In [ ]:
import cv2
from deepface import DeepFace
import speech_recognition as sr
from transformers import pipeline
import threading


def recognize_speech_from_mic():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        print("Listening for speech... (You can speak anytime)")
        recognizer.adjust_for_ambient_noise(source)

        while True:
            try:
                audio = recognizer.listen(source, timeout=5)
                print("Recognizing speech...")
                text = recognizer.recognize_google(audio)
                print(f"You said: {text}")
                emotions = detect_emotion_from_text(text)
                if emotions:
                    print("Detected emotions from speech:")
                    for emotion in emotions:
                        print(f"Emotion: {emotion['label']}, Score: {emotion['score']:.4f}")
            except sr.WaitTimeoutError:
                continue
            except sr.RequestError:
                print("Could not request results from Google Speech Recognition service.")
            except sr.UnknownValueError:
                print("Speech not understood. Try again.")


def detect_emotion_from_text(text):
    emotion_classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")
    emotions = emotion_classifier(text)
    return emotions


def detect_emotion_from_video():
    video_capture = cv2.VideoCapture(0)

    while True:
        ret, frame = video_capture.read()

        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        try:
            analysis = DeepFace.analyze(rgb_frame, actions=["emotion"], enforce_detection=False)

            dominant_emotion = analysis[0]["dominant_emotion"]

            cv2.putText(frame, dominant_emotion, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        except Exception as e:
            print("Error in emotion analysis (video):", e)

        cv2.imshow("Emotion Detection (Video)", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    video_capture.release()
    cv2.destroyAllWindows()


def main():
    video_thread = threading.Thread(target=detect_emotion_from_video, daemon=True)
    audio_thread = threading.Thread(target=recognize_speech_from_mic, daemon=True)

    video_thread.start()
    audio_thread.start()

    try:
        while True:
            pass
    except KeyboardInterrupt:
        print("\nStopping the program.")
    finally:
        cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
